In [1]:
import sys
sys.path.insert(0, '..')

import torch.nn as nn
from torch.utils.data import DataLoader
from utils.eye_dataset import *
from eye_classifier import *
import torchvision.transforms as transforms
import torchvision.models as models

### The Training Dataset

We use the utility class EyeImageDataset() to load the training dataset based on the metadata CSV file and
the target image folder

The training images will be loaded as needed due to memory constraints.

Since we're using RESNET-34 network as the convolutional layers, we need to resize our images to 224x224, so we apply
some transforms on our dataset.

We also need to proper normalize the dataset accordingly to the RESNET specification.

In [2]:
base_dir = "../../data"
image_dir = f"{base_dir}/preprocessed_images"

image_dir_training = f"{base_dir}/ODIR-5K/training"
image_dir_testing = f"{base_dir}/ODIR-5K/testing"
csv_file = f'{base_dir}/ODIR-5K/data.csv'

print ('reading input dataset')
input_size = 224

apply_transforms = transforms.Compose([
    transforms.Resize(size=input_size),
    transforms.CenterCrop(size=input_size),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

ds = EyeImageDataset(root=image_dir_training, data_info_csv_file=csv_file, transform=apply_transforms)


reading input dataset


### Building the model

To build the model, we use the utility superclass EyeClassifier. 

We create a subclass named ResnetEyeClassifier and then we feed it with the intended model on its __init__ constructor.

Here we're building a resnet34 as the first layer (in fact resnet18 is comprised of many layers), followed by three linear fully connected layers
for image classification

The last layer will not have a transfer function because we opted to use CrossEntropyLoss as the loss function. 

You can change the loss function by using set_loss_function() if you wish to

In [3]:
class ResnetEyeClassifier(EyeClassifier):
    def __init__(self, num_classes: int) -> None:
        super(ResnetEyeClassifier, self).__init__(model=[

            (models.resnet34(pretrained=True), TransferFunction.NotApplicable),

            (nn.Linear(in_features=1000, out_features=84),
             TransferFunction.LeakyRelu),

            (nn.Linear(in_features=84, out_features=42),
             TransferFunction.LeakyRelu),

            (nn.Linear(in_features=42, out_features=num_classes),
             TransferFunction.NotApplicable),
        ])


nn = ResnetEyeClassifier(num_classes=len(ds.classes))
print(nn)


ResnetEyeClassifier(
  (layer 1): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=Tru

### Training the model

Training the model is very easy. All we have to do is to call train_model() passing the EyeImageDataset object. 

If you're having I/O constraints, you can use the set_buffer_size(n) method to cache n images in memory. Be aware that you cant use it
along with shuffle, because the cache will be constantly invalidated, unless you have a lot of memory and cache all of the images.

In [4]:
nn.train_model(ds, batch_size=16, num_epochs=30)

training (1%) epoch 1/30, loss = 2.1852
training (2%) epoch 1/30, loss = 1.4304
training (3%) epoch 1/30, loss = 1.2855
training (4%) epoch 1/30, loss = 1.0888
training (5%) epoch 2/30, loss = 1.5553
training (6%) epoch 2/30, loss = 1.3558
training (7%) epoch 2/30, loss = 1.3807
training (8%) epoch 3/30, loss = 0.8107
training (9%) epoch 3/30, loss = 1.2432
training (10%) epoch 3/30, loss = 2.0691
training (11%) epoch 4/30, loss = 0.6668
training (12%) epoch 4/30, loss = 0.7239
training (13%) epoch 4/30, loss = 1.1934
training (14%) epoch 4/30, loss = 1.1669
training (15%) epoch 5/30, loss = 0.6158
training (16%) epoch 5/30, loss = 0.8835
training (17%) epoch 5/30, loss = 0.6047
training (18%) epoch 6/30, loss = 0.5347
training (19%) epoch 6/30, loss = 0.7634
training (20%) epoch 6/30, loss = 0.8200
training (21%) epoch 7/30, loss = 0.5979
training (22%) epoch 7/30, loss = 0.7596
training (23%) epoch 7/30, loss = 0.6771
training (24%) epoch 7/30, loss = 0.4447
training (25%) epoch 8/30

### Testing the model

Testing the model is similar to training it. All we have to do is to call test_model() passing a EyeImageDataset object pointing to the test dataset images. 

If you're having I/O constraints, you can use the set_buffer_size(n) method to cache n images in memory. Be aware that you cant use it
along with shuffle, because the cache will be constantly invalidated, unless you have a lot of memory and cache all of the images.

In [5]:
input_size = 224

apply_transforms = transforms.Compose([
    transforms.Resize(size=input_size),
    transforms.CenterCrop(size=input_size),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

ds = EyeImageDataset(root=image_dir_testing, data_info_csv_file=csv_file, transform=apply_transforms)


In [6]:
nn.test_model(ds)

testing 1% [4 / 6564 files]
testing 2% [72 / 6564 files]
testing 3% [140 / 6564 files]
testing 4% [208 / 6564 files]
testing 5% [276 / 6564 files]
testing 6% [344 / 6564 files]
testing 7% [412 / 6564 files]
testing 8% [480 / 6564 files]
testing 9% [548 / 6564 files]
testing 10% [616 / 6564 files]
testing 11% [684 / 6564 files]
testing 12% [752 / 6564 files]
testing 13% [820 / 6564 files]
testing 14% [888 / 6564 files]
testing 15% [956 / 6564 files]
testing 16% [1024 / 6564 files]
testing 17% [1092 / 6564 files]
testing 18% [1160 / 6564 files]
testing 19% [1228 / 6564 files]
testing 20% [1296 / 6564 files]
testing 21% [1364 / 6564 files]
testing 22% [1432 / 6564 files]
testing 23% [1500 / 6564 files]
testing 24% [1568 / 6564 files]
testing 25% [1636 / 6564 files]
testing 26% [1704 / 6564 files]
testing 27% [1772 / 6564 files]
testing 28% [1840 / 6564 files]
testing 29% [1908 / 6564 files]
testing 30% [1976 / 6564 files]
testing 31% [2044 / 6564 files]
testing 32% [2112 / 6564 files]
tes